In [1]:
using Distributions, Random, StatsBase,LinearAlgebra

In [2]:
function initialize_parameters(K, D)
    μ = [randn(D) for _ in 1:K]        # Means of clusters
    σ² = [rand(Gamma(2.0, 2.0)) for _ in 1:K]  # Variances
    π = normalize(rand(Dirichlet(1:K)))  # Mixture weights
    return μ, σ², π
end

initialize_parameters (generic function with 1 method)

In [3]:
μ, σ², π=initialize_parameters(3, 1)
σ²

3-element Vector{Float64}:
 9.070189081886982
 1.945365902273532
 4.372492330805952

In [8]:
function sample_assignments(R, μ, σ², π, K)
    N = length(R)
    z = zeros(Int, N)
    
    for t in 1:N
        #println("t=",t)
        #println("μ=",μ)
        #println("σ²=",σ²)
        #print("a=", pdf(Normal(μ[1][1], sqrt(σ²[1])),1))
        probs = [π[k] * pdf(Normal(μ[k][1], sqrt(σ²[k])),R[t]) for k in 1:K]
        #w=[Weights(probs[i]) for i in 1:K]
        w=Weights(probs)
        #println("w=",w)
        z[t] = sample(1:K, w)
    end
    
    return z
end

sample_assignments (generic function with 1 method)

In [4]:
bla=Normal(μ[1][1], sqrt(σ²[1]))
probs=[pdf(bla, R)
w=Weights(probs)
w=[Weights(probs[i]) for i in 1:3]
#sample(1:3, w[1])
w

Base.Meta.ParseError: ParseError:
# Error @ /Users/stirlitz/ncGitHub/daily_options/daily_options1/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W4sZmlsZQ==.jl:6:2
#sample(1:3, w[1])
w
#└ ── Expected `]`

In [5]:
function update_parameters(R, z, K, μ_0, λ_0, α_0, β_0)
    μ = Vector{Float64}(undef, K)
    σ² = Vector{Float64}(undef, K)
    n_k = counts(z, 1:K)  # Correctly counts points per cluster
    
    for k in 1:K
        assigned_points = R[z .== k]  # Points assigned to cluster k
        
        if n_k[k] > 0
            # Update parameters for cluster k
            R̄_k = mean(assigned_points)
            μ_post = (λ_0 * μ_0 + n_k[k] * R̄_k) / (λ_0 + n_k[k])
            λ_post = λ_0 + n_k[k]
            α_post = α_0 + n_k[k] / 2
            β_post = β_0 + sum((assigned_points .- R̄_k).^2) / 2 +
                      λ_0 * n_k[k] * (R̄_k - μ_0)^2 / (2 * λ_post)
            
            # Sample from posterior
            #println("σ²=",σ²)
            #println("k=$k, μ_post=$μ_post, λ_post=$λ_post, α_post=$α_post, β_post=$β_post, R̄_k=$R̄_k")
            σ²[k] = rand(InverseGamma(α_post, β_post))
            μ[k] = rand(Normal(μ_post, sqrt(σ²[k] / λ_post)))
        else
            # Use prior if no points are assigned to cluster k
            μ[k] = rand(Normal(μ_0, sqrt(1 / λ_0)))
            σ²[k] = rand(InverseGamma(α_0, β_0))
        end
    end
    
    # Sample new mixture weights using Dirichlet posterior
    π = rand(Dirichlet(n_k .+ 1))
    
    return μ, σ², π #, μ_post, λ_post, α_post, β_post
end

update_parameters (generic function with 1 method)

In [6]:
function gibbs_sampling(R, K, iterations)
    N = length(R)
    D = 1  # 1D data
    
    # Prior parameters
    μ_0 = 0.0
    λ_0 = 1.0
    α_0 = 2.0
    β_0 = 2.0
    println("Prior: μ_0 = $μ_0, λ_0 = $λ_0, α_0 = $α_0, β_0 = $β_0")

    # Initialize parameters
    μ, σ², π = initialize_parameters(K, D)
    #μ=[0,5]
    #σ²=[0,5]
    z = sample_assignments(R, μ, σ², π, K)
    println("Initial: Means = $μ, Variances = $σ², Weights = $π")
    println("Initial assignments: $z")

    # Perform Gibbs Sampling
    for iter in 1:iterations
        z = sample_assignments(R, μ, σ², π, K)
        μ, σ², π = update_parameters(R, z, K, μ_0, λ_0, α_0, β_0)
        
        println("Iteration $iter: Means = $μ, Variances = $σ², Weights = $π")
        #println("u_0=$μ_0, λ_0=$λ_0, α_0=$α_0, β_0=$β_0")
    end

    return μ, σ², π, z
end

gibbs_sampling (generic function with 1 method)

In [ ]:
# Generate synthetic data
using DataFrames
include("gibbs_sampling.jl")
true_means = [0.0, 0.0]
true_std = [1, 3]
true_variances = true_std.^2
R = vcat(rand(Normal(true_means[1], true_std[1]), 1000), rand(Normal(true_means[2], true_std[2]), 1000))
#Random.seed!(42)
z_true = vcat(ones(Int, 1000), 2*ones(Int, 1000))
means_sample=[mean(R[z_true.==1]), mean(R[z_true.==2])]
variances_sample=[var(R[z_true.==1]), var(R[z_true.==2])]
std_sample=sqrt.(variances_sample)

# Run Gibbs Sampling
K = 2   # Number of clusters
iterations = 1000
μ, σ², π, z = gibbs_sampling(R, K, iterations)
σ=sqrt.(σ²)
sample_cluster_means=[]
println("Means = $μ, Variances = $σ, Weights = $π,  true means = [0, 0.0], true variances = [0.01, 0.03]")
println("true means: $true_means, true variances: $true_variances")

#df_summary=DataFrame(iteration=1:iterations, μ=μ, σ=σ, π=π)



Prior: μ_0 = 0.0, λ_0 = 1.0, α_0 = 2.0, β_0 = 2.0
Means = [-0.041602628984714146, 0.14790233402293687], Variances = [1.1291827683260565, 3.0576819726926745], Weights = [0.545549798036791, 0.45445020196320896],  true means = [0, 0.0], true variances = [0.01, 0.03]
true means: [0.0, 0.0], true variances: [1, 9]


DimensionMismatch: DimensionMismatch: column :iteration has length 1000 and column :μ has length 2

In [21]:
std(R[1:1000])

1.0163093482362437

In [23]:
var(R)
σ²[1]*π[1]+σ²[2]*π[2]+σ²[3]*π[3]
#σ²[2]
#var_theory=25*.5+1*.5

13.347926719671301

In [49]:
K=3
probs = [π[k] * pdf(Normal(μ[k][1], sqrt(σ²[k])), 1) for k in 1:K]
w=Weights(probs)
#w=[.1,.2,.7]
#sample(1:K, w)

#sample(1:K, Weights(w))
# Example of using the sample method with generated inputs
#generated_probs = [0.2, 0.5, 0.3]
#generated_weights = Weights(generated_probs)
sample(1:3, w)

2

In [19]:
z

200-element Vector{Int64}:
 2
 2
 2
 2
 1
 2
 1
 1
 2
 2
 ⋮
 2
 2
 2
 1
 2
 2
 1
 2
 2